# Grasp annotation example

In [ ]:
from pathlib import Path
import cv2
from airo_camera_toolkit.point_clouds.conversions import point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud
from cloth_tools.dataset.format import load_competition_observation
import matplotlib.pyplot as plt
from airo_camera_toolkit.utils.image_converter import ImageConverter
import open3d as o3d

data_dir = Path("../data")
dataset_dir = data_dir / "cloth_competition_dataset_0000"
sample_dir = dataset_dir / "sample_000000"

In [ ]:
observation_start_dir = sample_dir / "observation_start"

observation = load_competition_observation(observation_start_dir)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(observation.image_left)

In [ ]:
confidence_map = observation.confidence_map
point_cloud = observation.point_cloud

# Filter outs point with low depth confidence (i.e. with high value in the confidence map)
confidence_threshold = 1.0
confidence_mask = (confidence_map <= confidence_threshold).reshape(-1)  # Threshold and flatten
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)
pcd_filtered = point_cloud_to_open3d(point_cloud_filtered)
pcd_filtered.point.positions.dtype, pcd_filtered.point.colors.dtype

In [ ]:
from cloth_tools.annotation.grasp_annotation import top_down_camera_pose

virtual_camera_pose = top_down_camera_pose(height=1.5)

Running the cell below will show the filtered point cloud in open3d. Press 'q' to close the window.

In [ ]:
from cloth_tools.visualization.open3d import open3d_camera

color_frontal_rgb = (1, 1, 0)
color_topdown_rgb = (0, 1, 1)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)

# Visualize the cameras
resolution = observation.camera_resolution
intrinsics = observation.camera_intrinsics

X_W_VC = virtual_camera_pose
X_W_C = observation.camera_pose_in_world
camera_frontal_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
camera_frontal_frame.transform(X_W_C)
camera_top_down_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
camera_top_down_frame.transform(X_W_VC)
camera_frontal_lines = open3d_camera(X_W_C, intrinsics, resolution, color_frontal_rgb, scale=0.2)
camera_topdown_lines = open3d_camera(X_W_VC, intrinsics, resolution, color_topdown_rgb, scale=0.2)

o3d.visualization.draw_geometries(
    [
        pcd_filtered.to_legacy(),
        world_frame,
        camera_frontal_frame,
        camera_top_down_frame,
        camera_frontal_lines,
        camera_topdown_lines,
    ]
)

In [ ]:
from cloth_tools.annotation.grasp_annotation import project_point_cloud_to_image

image_topdown = project_point_cloud_to_image(
    point_cloud_filtered, X_W_VC, intrinsics, resolution, background_color=(90, 90, 90)
)

# Matplotlib seems to do some anti-aliasing, which makes the image look better than in opencv (without blurring)
plt.figure(figsize=(12, 6))
plt.imshow(ImageConverter.from_opencv_format(image_topdown).image_in_numpy_int_format)
plt.show()

In [ ]:
from cloth_tools.annotation.grasp_annotation import get_manual_grasp_annotation

grasp_info = get_manual_grasp_annotation(
    observation.image_left, observation.depth_map, point_cloud_filtered, X_W_C, intrinsics, log_to_rerun=True
)

In [ ]:
grasp_info.__dict__.keys()

In [ ]:
grasp_info.grasp_pose

In [ ]:
from cloth_tools.annotation.grasp_annotation import GraspAnnotationInfo, save_grasp_info
from airo_dataset_tools.data_parsers.pose import Pose
import json
import os

grasp_dir = Path("grasp_new")
save_grasp_info(grasp_dir, grasp_info)

In [ ]:
from cloth_tools.annotation.grasp_annotation import GraspAnnotation

grasp_annotation_file = grasp_dir / "grasp_annotation.json"

with open(grasp_annotation_file, "r") as f:
    grasp_annotation = GraspAnnotation.model_validate_json(f.read())

grasp_annotation

In [ ]:
!rm -rf grasp_new